In [2]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import random

# Load the CSV file containing image file names and class labels
df = pd.read_csv('/kaggle/input/2d-fetal-altrasound-images/Classification/image_label.csv')

# Define constants for image resizing and normalization
image_width = 128
image_height = 128
image_channels = 3

# Initialize empty lists to store preprocessed data
images = []
labels = []

# Iterate through each row in the CSV file and load/encode images and labels
for index, row in df.iterrows():
    image_path = f"/kaggle/input/2d-fetal-altrasound-images/Classification/images/{row['Image_name']}.png"
    label = row['Plane']
    
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_width, image_height))
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    
    # Append the preprocessed image and encoded label to the lists
    images.append(image)
    labels.append(label)

# Encode class labels into numerical values
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
label_map = {}
for i, x in enumerate(labels_encoded):
    if x not in label_map:
        label_map[x] = labels[i]

In [6]:
label_map

{1: 'Fetal brain', 2: 'Fetal femur', 3: 'Fetal thorax', 0: 'Fetal abdomen'}

In [3]:
# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,  # Randomly rotate images by up to 10 degrees  # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Randomly zoom in by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill in missing pixels with the nearest value
)

In [4]:
# Augmentation
augmented_images = []
augmented_labels = []

for i, (image, label) in enumerate(zip(images, labels_encoded)):
    image = np.expand_dims(image, axis=0)
    label = np.array([label])
    
    # Generate augmented images and labels
    amount = 0
    for batch in datagen.flow(image, label, batch_size=1):
        augmented_image, augmented_label = batch
        augmented_image = augmented_image[0]
        augmented_label = augmented_label[0]
        
        augmented_images.append(augmented_image)
        augmented_labels.append(augmented_label)
        
        amount += 1
        if amount >= random.choice([5,6,7,8,9,10]):  # You can adjust this number
            break

# Append augmented data to the original data sets
images.extend(augmented_images)
labels_encoded = np.concatenate([labels_encoded, augmented_labels])

In [8]:
# Split the dataset into training, validation, and testing sets (80-10-10 split)
X_train, X_temp, y_train, y_temp = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Convert the extended data to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Check the shapes of the data splits
print("Training data shape:", X_train.shape)
print("Validation data shape:", X_val.shape)
print("Testing data shape:", X_test.shape)
print("Training label shape:", y_train.shape)
print("Validation label shape:", y_val.shape)
print("Testing label shape:", y_test.shape)

Training data shape: (10236, 128, 128, 3)
Validation data shape: (1280, 128, 128, 3)
Testing data shape: (1280, 128, 128, 3)
Training label shape: (10236,)
Validation label shape: (1280,)
Testing label shape: (1280,)


In [9]:
num_classes = 4

# Convert labels to one-hot encoded format
y_train_onehot = to_categorical(y_train, num_classes)
y_val_onehot = to_categorical(y_val, num_classes)
y_test_onehot = to_categorical(y_test, num_classes)

## *Model Training*

In [14]:
# Build a Neural Network Model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(16, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the Model
history = model.fit(X_train, y_train_onehot, epochs=10, validation_data=(X_val, y_val_onehot), callbacks=[early_stopping])

# Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test_onehot)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
320/320 [==============================] - 17s 19ms/step - loss: 0.6405 - accuracy: 0.7683 - val_loss: 0.4729 - val_accuracy: 0.8398
Epoch 2/10
320/320 [==============================] - 4s 14ms/step - loss: 0.3058 - accuracy: 0.9003 - val_loss: 0.3427 - val_accuracy: 0.8766
Epoch 3/10
320/320 [==============================] - 4s 13ms/step - loss: 0.1634 - accuracy: 0.9448 - val_loss: 0.3101 - val_accuracy: 0.8984
Epoch 4/10
320/320 [==============================] - 4s 12ms/step - loss: 0.0842 - accuracy: 0.9725 - val_loss: 0.3140 - val_accuracy: 0.8984
Epoch 5/10
320/320 [==============================] - 4s 13ms/step - loss: 0.0357 - accuracy: 0.9900 - val_loss: 0.3862 - val_accuracy: 0.9062
Epoch 6/10
40/40 [==============================] - 1s 7ms/step - loss: 0.2667 - accuracy: 0.9086
Test accuracy: 0.9085937738418579


In [15]:
# Save the trained model to a file
model.save('model.h5')

## *Inference*

In [21]:
# Load the saved model
model = keras.models.load_model('model.h5')

images = []
classes = []

# Image directory
dir_path = '/kaggle/input/2d-fetal-altrasound-images/Classification/External Test images'

for file_name in os.listdir(dir_path):
    
    image_path = dir_path+"/"+file_name
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_width, image_height))  # Resize the image to match your model's input size
    image = image / 255.0
    images.append(image)
    # Make a prediction on the image
    predictions = model.predict(np.expand_dims(image, axis=0))

    # Decode the predictions (if you one-hot encoded your labels)
    predicted_class = np.argmax(predictions)

    # Map the predicted class back to your class names
    predicted_class_name = label_map[predicted_class]
    classes.append(predicted_class_name)
    print(f'Predicted class: {predicted_class_name}')

1/1 [==============================] - 0s 81ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 22ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 20ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 20ms/step
Predicted class: Fetal abdomen
1/1 [==============================] - 0s 21ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 21ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 21ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 21ms/step
Predicted class: Fetal brain
1/1 [==============================] - 0s 22ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 20ms/step
Predicted class: Fetal femur
1/1 [==============================] - 0s 21ms/step
Predicted class: Fetal thorax
1/1 [==============================] - 0s 21ms/step
Predicted class: Fetal femur
1/1 [================

In [31]:
# You can adjust the number of rows and columns as per your preference
num_rows = 8
num_cols = 5
image_spacing = 40
# Calculate the dimensions of the collage with spacing
image_height, image_width, _ = images[0].shape
collage_width = (image_width + image_spacing) * num_cols
collage_height = (image_height + image_spacing) * num_rows

# Create an empty canvas for the collage
collage = np.zeros((collage_height, collage_width, 3), dtype=np.uint8)

# Fill the canvas with new sample images and their predicted classes
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.6
font_thickness = 2
font_color = (255, 255, 255)  # White text color

for i in range(num_rows):
    for j in range(num_cols):
        index = i * num_cols + j
        if index < len(images):
            sample_image = (images[index] * 255.0).astype(np.uint8)  # Convert to 8-bit
            y_offset = i * (image_height + image_spacing)
            x_offset = j * (image_width + image_spacing)
            collage[y_offset:y_offset + image_height, x_offset:x_offset + image_width] = sample_image

            # Add the predicted class label
            class_label = classes[index]
            text_size, _ = cv2.getTextSize(class_label, font, font_scale, font_thickness)
            text_x = x_offset + (sample_image.shape[1] - text_size[0]) // 2
            text_y = y_offset + sample_image.shape[0] + text_size[1] + 5
            cv2.putText(collage, class_label, (text_x, text_y), font, font_scale, font_color, font_thickness)

cv2.imwrite('Image_with_Prediction.jpg', collage)

True